# SHAP Report - PISA 2022 Amazon SageMaker XGBoost
_**This Notebook Runs a SHAP report to explain variables on our model**_

---

---

## Contents

1. [Background](#Background)
1. [Prepration](#Preparation)
1. [Data](#Data)
    1. [Exploration](#Exploration)
    1. [Transformation](#Transformation)
1. [Training](#Training)
1. [Hosting](#Hosting)
1. [Evaluation](#Evaluation)
1. [Exentsions](#Extensions)

---

## Background

This notebook runs an Amazon SageMaker pipeline to predict if students will fall behind in Math using hte PISA 2022 dataset

* Preparing your Amazon SageMaker notebook
* Downloading data from the internet into Amazon SageMaker
* Investigating and transforming the data so that it can be fed to Amazon SageMaker algorithms
* Estimating a model using the Gradient Boosting algorithm
* Evaluating the effectiveness of the model
* Setting the model up to make on-going predictions

---

## Preparation

_This notebook was created and tested on an ml.m4.xlarge notebook instance._

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp with a the appropriate full IAM role arn string(s).

In [10]:
# cell 02
import sagemaker
bucket=sagemaker.Session().default_bucket()
prefix = 'sagemaker/DEMO-xgboost-no-wle-dm-1500-samples'
 
# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

Now let's bring in the Python libraries that we'll use throughout the analysis

In [11]:
# cell 03
import numpy as np                                # For matrix operations and numerical processing
import pandas as pd                               # For munging tabular data
import matplotlib.pyplot as plt                   # For charts and visualizations
from IPython.display import Image                 # For displaying images in the notebook
from IPython.display import display               # For displaying outputs in the notebook
from time import gmtime, strftime                 # For labeling SageMaker models, endpoints, etc.
import sys                                        # For writing outputs to notebook
import math                                       # For ceiling function
import json                                       # For parsing hosting outputs
import os                                         # For manipulating filepath names
import sagemaker 
import zipfile     # Amazon SageMaker's Python SDK provides many helper functions
import mlflow

In [12]:
# cell 04
pd.__version__

'2.2.3'

Make sure pandas version is set to 1.2.4 or later. If it is not the case, restart the kernel before going further

---

## Download PISA 2022 Prepared Dataset

This is our dataset output from our cleaning notebook [here](https://7z4vtvpqcoxouiu.studio.us-west-2.sagemaker.aws/jupyterlab/default/lab/tree/RTC%3Amids-capstone/notebooks/eda/Data_merging.ipynb)


In [4]:
%%time 

# cell 06

# Define local file path
local_file_path = "PISA_cleaned_dataset.csv"  # Change as needed

# Define S3 details
bucket_name = "sagemaker-us-west-2-986030204467"
file_key = "capstone/testfiles/PISA_cleaned_dataset.csv"

# Check if the file exists locally
if os.path.exists(local_file_path):
    print("📂 Loading data from local file...")
    data = pd.read_csv(local_file_path, usecols=None)
else:
    print("☁️ Downloading data from S3...")
    
    # Create S3 client
    s3_client = boto3.client("s3")

    # Download the file from S3
    response = s3_client.get_object(Bucket=bucket_name, Key=file_key)

    # Read the file into pandas DataFrame
    data = pd.read_csv(response["Body"], usecols=None)

    # Save a local copy for future use
    data.to_csv(local_file_path, index=False)
    print(f"✅ File saved locally as {local_file_path}")

# Display first few rows
#data.head()

pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 20)         # Keep the output on one page
data

📂 Loading data from local file...
CPU times: user 24.5 s, sys: 6.92 s, total: 31.5 s
Wall time: 31.6 s


,CNT,CNTSCHID,CNTSTUID,MATH_Proficient,SISCO,ST347Q01JA,ST347Q02JA,ST349Q01JA_0,ST349Q01JA_1,ST349Q01JA_2,ST349Q01JA_3,ST349Q01JA_4,ST350Q01JA,ST356Q01JA,ST322Q01JA,ST322Q02JA,ST322Q03JA,ST322Q04JA,ST322Q06JA,ST322Q07JA,DURECEC,EFFORT1,EFFORT2,ST259Q01JA,WB164Q01HA,HOMEPOS,ST004D01T,GRADE,REPEAT,EXPECEDU,ICTAVSCH,ICTAVHOM,ICTDISTR,IMMIG,TARDYSD,ST226Q01JA,ST016Q01NA,MISSSC,Option_UH,OECD,PAREDINT,BMMJ1,BFMJ2,WB163Q06HA,WB163Q07HA,ST230Q01JA,SKIPPING,IC180Q01JA,IC180Q08JA,ST059Q02JA,ST296Q04JA,WB176Q01HA,STUDYHMW,IC184Q01JA,IC184Q02JA,IC184Q03JA,IC184Q04JA,ST059Q01TA,ST296Q01JA,ST272Q01JA,ST268Q01JA,ST268Q04JA,ST268Q07JA,ST293Q04JA,ST297Q01JA,ST297Q03JA,ST297Q05JA,ST297Q06JA,ST297Q07JA,ST297Q09JA,WB165Q01HA,WB166Q01HA,WB166Q02HA,WB166Q03HA,WB166Q04HA,ST258Q01JA,ST294Q01JA,ST295Q01JA,WB150Q01HA,WB156Q01HA,WB158Q01HA,WB160Q01HA,WB161Q01HA,WB171Q01HA,WB171Q02HA,WB171Q03HA,WB171Q04HA,WB172Q01HA,WB173Q01HA,WB173Q02HA,WB173Q03HA,WB173Q04HA,WB177Q01HA,WB177Q02HA,WB177Q03HA,WB177Q04HA,WB032Q01NA,WB032Q02NA,WB031Q01NA,EXERPRAC,STUBMI,RELATST,BELONG,BULLIED,FEELSAFE,SCHRISK,PERSEVAGR,CURIOAGR,COOPAGR,EMPATAGR,ASSERAGR,STRESAGR,EMOCOAGR,GROSAGR,INFOSEEK,FAMSUP,DISCLIM,TEACHSUP,COGACRCO,COGACMCO,EXPOFA,EXPO21ST,MATHEFF,MATHEF21,FAMCON,ANXMAT,MATHPERS,CREATEFF,CREATSCH,CREATFAM,CREATAS,CREATOOS,CREATOP,OPENART,IMAGINE,SCHSUST,LEARRES,PROBSELF,FAMSUPSL,FEELLAH,SDLEFF,ICTRES,ESCS,FLSCHOOL,FLMULTSB,FLFAMILY,ACCESSFP,FLCONFIN,FLCONICT,ACCESSFA,ATTCONFM,FRINFLFM,ICTSCH,ICTHOME,ICTQUAL,ICTSUBJ,ICTENQ,ICTFEED,ICTOUT,ICTWKDY,ICTWKEND,ICTREG,ICTINFO,ICTEFFIC,BODYIMA,SOCONPA,LIFESAT,PSYCHSYM,SOCCON,EXPWB,CURSUPP,PQMIMP,PQMCAR,PARINVOL,PQSCHOOL,PASCHPOL,ATTIMMP,CREATHME,CREATACT,CREATOPN,CREATOR,WORKPAY,WORKHOME,SC001Q01TA,SC211Q01JA,SC211Q02JA,SC211Q03JA,SC211Q04JA,SC211Q05JA,SC211Q06JA,SC209Q04JA,SC209Q05JA,SC209Q06JA,SC037Q11JA,SC183Q02JA,SC183Q03JA,SC183Q04JA,SC175Q01JA,SC177Q01JA_1,SC177Q01JA_2,SC177Q01JA_3,SC177Q02JA_1,SC177Q02JA_2,SC177Q02JA_3,SC177Q03JA_1,SC177Q03JA_2,SC177Q03JA_3,SC188Q01JA,SC188Q02JA,SC188Q03JA,SC188Q04JA,SC188Q05JA,SC188Q06JA,SC188Q07JA,SC188Q08JA,SC188Q09JA,SC188Q10JA,SC188Q11JA,SC198Q01JA,SC198Q02JA,SC198Q03JA,SC178Q01JA,SC178Q02JA,SC180Q01JA,SC189Q02WA,SC189Q03WA,SC189Q04WA,SMRATIO,MCLSIZE,MACTIV,MATHEXC_0,MATHEXC_1,MATHEXC_2,MATHEXC_3,ABGMATH,SC064Q05WA,SC064Q06WA,SC064Q01TA,SC064Q02TA,SC064Q04NA,SC064Q03TA,SC064Q07WA,SC213Q01JA,SC213Q02JA,SC037Q01TA,SC037Q02TA,SC037Q03TA,SC037Q04TA,SC037Q05NA,SC037Q06NA,...,DIGDVPOL,TEAFDBK,MTTRAIN,DMCVIEWS,NEGSCLIM,STAFFSHORT,EDUSHORT,STUBEHA,TEACHBEHA,STDTEST,TDTEST,ALLACTIV,BCREATSC,CREENVSC,ACTCRESC,OPENCUL,PROBSCRI,SCPREPBP,SCPREPAP,DIGPREP,LANGN_105,LANGN_108,LANGN_112,LANGN_113,LANGN_118,LANGN_121,LANGN_130,LANGN_133,LANGN_137,LANGN_140,LANGN_147,LANGN_148,LANGN_150,LANGN_154,LANGN_156,LANGN_160,LANGN_170,LANGN_195,LANGN_200,LANGN_202,LANGN_204,LANGN_232,LANGN_237,LANGN_244,LANGN_246,LANGN_254,LANGN_258,LANGN_263,LANGN_264,LANGN_266,LANGN_272,LANGN_273,LANGN_275,LANGN_286,LANGN_301,LANGN_313,LANGN_316,LANGN_317,LANGN_322,LANGN_325,LANGN_327,LANGN_329,LANGN_338,LANGN_340,LANGN_344,LANGN_351,LANGN_358,LANGN_363,LANGN_369,LANGN_371,LANGN_375,LANGN_379,LANGN_381,LANGN_382,LANGN_383,LANGN_404,LANGN_409,LANGN_415,LANGN_420,LANGN_422,LANGN_428,LANGN_434,LANGN_442,LANGN_449,LANGN_451,LANGN_463,LANGN_465,LANGN_467,LANGN_471,LANGN_472,LANGN_474,LANGN_492,LANGN_493,LANGN_494,LANGN_495,LANGN_496,LANGN_500,LANGN_503,LANGN_514,LANGN_517,LANGN_520,LANGN_523,LANGN_527,LANGN_529,LANGN_531,LANGN_540,LANGN_547,LANGN_555,LANGN_561,LANGN_562,LANGN_563,LANGN_565,LANGN_566,LANGN_567,LANGN_600,LANGN_601,LANGN_602,LANGN_605,LANGN_606,LANGN_607,LANGN_608,LANGN_611,LANGN_614,LANGN_615,LANGN_616,LANGN_618,LANGN_619,LANGN_621,LANGN_622,LANGN_623,LANGN_624,LANGN_625,LANGN_626,LANGN_627,LANGN_628,LANGN_630,LANGN_631,LANGN_634,LANGN_635,LANGN_639,LANGN_640,LANGN_641,LANGN_642,LANGN_648,LANGN_650,LANGN_661,LANGN_662,LANGN_663,LANGN_665,LANGN_666,LANGN_667,LANGN_668,LANGN_669,LANGN_670,LANGN_673,LANGN_674,LANGN_

# REMOVE WLE VARIABLES

In [5]:
# Define the list of columns to drop
wle_columns_to_remove = [
    "HOMEPOS", "RELATST", "BELONG", "BULLIED", "FEELSAFE", "SCHRISK", "PERSEVAGR", "CURIOAGR", 
    "COOPAGR", "EMPATAGR", "ASSERAGR", "STRESAGR", "EMOCOAGR", "GROSAGR", "INFOSEEK", "FAMSUP", 
    "DISCLIM", "TEACHSUP", "COGACRCO", "COGACMCO", "EXPOFA", "EXPO21ST", "MATHEFF", "MATHEF21", 
    "FAMCON", "ANXMAT", "MATHPERS", "CREATEFF", "CREATSCH", "CREATFAM", "CREATAS", "CREATOOS", 
    "CREATOP", "OPENART", "IMAGINE", "SCHSUST", "LEARRES", "PROBSELF", "FAMSUPSL", "FEELLAH", 
    "SDLEFF", "ICTRES", "FLSCHOOL", "FLMULTSB", "FLFAMILY", "ACCESSFP", "FLCONFIN", "FLCONICT", 
    "ACCESSFA", "ATTCONFM", "FRINFLFM", "ICTSCH", "ICTHOME", "ICTQUAL", "ICTSUBJ", "ICTENQ", 
    "ICTFEED", "ICTOUT", "ICTWKDY", "ICTWKEND", "ICTREG", "ICTINFO", "ICTEFFIC", "BODYIMA", 
    "SOCONPA", "LIFESAT", "PSYCHSYM", "SOCCON", "EXPWB", "CURSUPP", "PQMIMP", "PQMCAR", 
    "PARINVOL", "PQSCHOOL", "PASCHPOL", "ATTIMMP", "CREATHME", "CREATACT", "CREATOPN", 
    "CREATOR", "SCHAUTO", "TCHPART", "EDULEAD", "INSTLEAD", "ENCOURPG", "DIGDVPOL", "TEAFDBK", 
    "MTTRAIN", "DMCVIEWS", "NEGSCLIM", "STAFFSHORT", "EDUSHORT", "STUBEHA", "TEACHBEHA", 
    "STDTEST", "TDTEST", "ALLACTIV", "BCREATSC", "CREENVSC", "ACTCRESC", "OPENCUL", 
    "PROBSCRI", "SCPREPBP", "SCPREPAP", "DIGPREP"
]

# Drop the columns
data = data.drop(columns=wle_columns_to_remove, errors='ignore')  # `errors='ignore'` prevents errors if a column isn't found

# Check the shape after dropping
print("Shape after dropping WLE columns:", data.shape)

Shape after dropping WLE columns: (591857, 465)


Let's talk about the data.  At a high level, we can see:

_**Specifics on each of the features:**_

*Target variable:*
* `MATH_Proficient`: Is the student proficient in Math per PISA statistics? (binary: 'yes','no')

### Exploration
Let's start exploring the data in our data prep widget.  First, let's understand how the features are distributed.

In [6]:
print(data['MATH_Proficient'].shape)

(591857,)


In [7]:
print(data.columns.duplicated().any()) 

False


In [8]:
# cell 10
# Convert categorical variables to sets of indicators
# model_data = pd.get_dummies(data, dtype=float)   
model_data = data

Another question to ask yourself before building a model is whether certain features will add value in your final use case.  For example, if your goal is to deliver the best prediction, then will you have access to that data at the moment of prediction?  Knowing it's raining is highly predictive for umbrella sales, but forecasting weather far enough out to plan inventory on umbrellas is probably just as difficult as forecasting umbrella sales without knowledge of the weather.  So, including this in your model may give you a false sense of precision.

Following this logic, let's remove the economic features and `duration` from our data as they would need to be forecasted with high precision to use as inputs in future predictions.

Even if we were to use values of the economic indicators from the previous quarter, this value is likely not as relevant for prospects contacted early in the next quarter as those contacted later on.

When building a model whose primary goal is to predict a target value on new data, it is important to understand overfitting.  Supervised learning models are designed to minimize error between their predictions of the target value and actuals, in the data they are given.  This last part is key, as frequently in their quest for greater accuracy, machine learning models bias themselves toward picking up on minor idiosyncrasies within the data they are shown.  These idiosyncrasies then don't repeat themselves in subsequent data, meaning those predictions can actually be made less accurate, at the expense of more accurate predictions in the training phase.

The most common way of preventing this is to build models with the concept that a model shouldn't only be judged on its fit to the data it was trained on, but also on "new" data.  There are several different ways of operationalizing this, holdout validation, cross-validation, leave-one-out validation, etc.  For our purposes, we'll simply randomly split the data into 3 uneven groups.  The model will be trained on 70% of data, it will then be evaluated on 20% of data to give us an estimate of the accuracy we hope to have on "new" data, and 10% will be held back as a final testing dataset which will be used later on.

In [9]:
# cell 12
train_data, validation_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data)), int(0.9 * len(model_data))])   # Randomly sort the data then split out first 70%, second 20%, and last 10%

/opt/conda/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Amazon SageMaker's XGBoost container expects data in the libSVM or CSV data format.  For this example, we'll stick to CSV.  Note that the first column must be the target variable and the CSV should not include headers.  Also, notice that although repetitive it's easiest to do this after the train|validation|test split rather than before.  This avoids any misalignment issues due to random reordering.

In [13]:
# cell 13
#pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
#pd.concat([validation_data['y_yes'], validation_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('validation.csv', index=False, header=False)# Drop non-numeric columns (e.g., country names or IDs that are not numeric)
non_numeric_columns = train_data.select_dtypes(exclude=['number']).columns
train_data = train_data.drop(columns=non_numeric_columns)
validation_data = validation_data.drop(columns=non_numeric_columns)

# Save train dataset with MATH_Proficient as the first column
train_data[['MATH_Proficient'] + [col for col in train_data.columns if col != 'MATH_Proficient']].to_csv('train.csv', index=False, header=False)

# Save validation dataset with MATH_Proficient as the first column
validation_data[['MATH_Proficient'] + [col for col in validation_data.columns if col != 'MATH_Proficient']].to_csv('validation.csv', index=False, header=False)


In [14]:
train_data = pd.concat([train_data["MATH_Proficient"], train_data.drop(["MATH_Proficient"], axis=1)], axis=1)
validation_data = pd.concat([validation_data["MATH_Proficient"], validation_data.drop(["MATH_Proficient"], axis=1)], axis=1)
test_data = pd.concat([test_data["MATH_Proficient"], test_data.drop(["MATH_Proficient"], axis=1)], axis=1)

In [15]:
train_data.head(5)

,MATH_Proficient,CNTSCHID,CNTSTUID,SISCO,ST347Q01JA,ST347Q02JA,ST349Q01JA_0,ST349Q01JA_1,ST349Q01JA_2,ST349Q01JA_3,ST349Q01JA_4,ST350Q01JA,ST356Q01JA,ST322Q01JA,ST322Q02JA,ST322Q03JA,ST322Q04JA,ST322Q06JA,ST322Q07JA,DURECEC,EFFORT1,EFFORT2,ST259Q01JA,WB164Q01HA,ST004D01T,GRADE,REPEAT,EXPECEDU,ICTAVSCH,ICTAVHOM,ICTDISTR,IMMIG,TARDYSD,ST226Q01JA,ST016Q01NA,MISSSC,Option_UH,OECD,PAREDINT,BMMJ1,BFMJ2,WB163Q06HA,WB163Q07HA,ST230Q01JA,SKIPPING,IC180Q01JA,IC180Q08JA,ST059Q02JA,ST296Q04JA,WB176Q01HA,STUDYHMW,IC184Q01JA,IC184Q02JA,IC184Q03JA,IC184Q04JA,ST059Q01TA,ST296Q01JA,ST272Q01JA,ST268Q01JA,ST268Q04JA,ST268Q07JA,ST293Q04JA,ST297Q01JA,ST297Q03JA,ST297Q05JA,ST297Q06JA,ST297Q07JA,ST297Q09JA,WB165Q01HA,WB166Q01HA,WB166Q02HA,WB166Q03HA,WB166Q04HA,ST258Q01JA,ST294Q01JA,ST295Q01JA,WB150Q01HA,WB156Q01HA,WB158Q01HA,WB160Q01HA,WB161Q01HA,WB171Q01HA,WB171Q02HA,WB171Q03HA,WB171Q04HA,WB172Q01HA,WB173Q01HA,WB173Q02HA,WB173Q03HA,WB173Q04HA,WB177Q01HA,WB177Q02HA,WB177Q03HA,WB177Q04HA,WB032Q01NA,WB032Q02NA,WB031Q01NA,EXERPRAC,STUBMI,ESCS,WORKPAY,WORKHOME,SC001Q01TA,SC211Q01JA,SC211Q02JA,SC211Q03JA,SC211Q04JA,SC211Q05JA,SC211Q06JA,SC209Q04JA,SC209Q05JA,SC209Q06JA,SC037Q11JA,SC183Q02JA,SC183Q03JA,SC183Q04JA,SC175Q01JA,SC177Q01JA_1,SC177Q01JA_2,SC177Q01JA_3,SC177Q02JA_1,SC177Q02JA_2,SC177Q02JA_3,SC177Q03JA_1,SC177Q03JA_2,SC177Q03JA_3,SC188Q01JA,SC188Q02JA,SC188Q03JA,SC188Q04JA,SC188Q05JA,SC188Q06JA,SC188Q07JA,SC188Q08JA,SC188Q09JA,SC188Q10JA,SC188Q11JA,SC198Q01JA,SC198Q02JA,SC198Q03JA,SC178Q01JA,SC178Q02JA,SC180Q01JA,SC189Q02WA,SC189Q03WA,SC189Q04WA,SMRATIO,MCLSIZE,MACTIV,MATHEXC_0,MATHEXC_1,MATHEXC_2,MATHEXC_3,ABGMATH,SC064Q05WA,SC064Q06WA,SC064Q01TA,SC064Q02TA,SC064Q04NA,SC064Q03TA,SC064Q07WA,SC213Q01JA,SC213Q02JA,SC037Q01TA,SC037Q02TA,SC037Q03TA,SC037Q04TA,SC037Q05NA,SC037Q06NA,SC037Q07TA,SC037Q09TA,SC200Q01JA,SC200Q02JA,SC200Q03JA,SC200Q04JA,SC224Q01JA,RATCMP1,RATCMP2,RATTAB,SCHSEL,SCHLTYPE_1,SCHLTYPE_2,SCHLTYPE_3,SC034Q01NA,SC034Q02NA,SC034Q03TA,SC034Q04TA,SC195Q01JA,SC195Q02JA,SC195Q03JA,SC195Q04JA,SC042Q01TA,SC042Q02TA,SC214Q01JA,SC214Q02JA,SC214Q03JA,SC215Q01JA,SC215Q02JA,SC215Q03JA,SC215Q04JA,SC215Q05JA,SC215Q06JA,SC215Q07JA,SC215Q08JA,SC216Q06JA,SC216Q07JA,SC216Q08JA,SC216Q09JA,SC217Q01JA,SC217Q02JA,SC217Q03JA,SC217Q04JA,SC217Q05JA,SC217Q06JA,SC217Q07JA,SC217Q08JA,SC217Q10JA,SC218Q01JA,SC219Q01JA,SC220Q01JA,SC221Q01JA,SC221Q02JA,SC221Q03JA,SC221Q04JA,SCSUPRTED,SCSUPRT,SC212Q01JA,SC212Q02JA,SC212Q03JA,SC037Q08TA,SC032Q01TA,SC032Q02TA,SC032Q03TA,SC032Q04TA,LANGN_105,LANGN_108,LANGN_112,LANGN_113,LANGN_118,LANGN_121,LANGN_130,LANGN_133,LANGN_137,LANGN_140,LANGN_147,LANGN_148,LANGN_150,LANGN_154,LANGN_156,LANGN_160,LANGN_170,LANGN_195,LANGN_200,LANGN_202,LANGN_204,LANGN_232,LANGN_237,LANGN_244,LANGN_246,LANGN_254,LANGN_258,LANGN_263,LANGN_264,LANGN_266,LANGN_272,LANGN_273,LANGN_275,LANGN_286,LANGN_301,LANGN_313,LANGN_316,LANGN_317,LANGN_322,LANGN_325,LANGN_327,LANGN_329,LANGN_338,LANGN_340,LANGN_344,LANGN_351,LANGN_358,LANGN_363,LANGN_369,LANGN_371,LANGN_375,LANGN_379,LANGN_381,LANGN_382,LANGN_383,LANGN_404,LANGN_409,LANGN_415,LANGN_420,LANGN_422,LANGN_428,LANGN_434,LANGN_442,LANGN_449,LANGN_451,LANGN_463,LANGN_465,LANGN_467,LANGN_471,LANGN_472,LANGN_474,LANGN_492,LANGN_493,LANGN_494,LANGN_495,LANGN_496,LANGN_500,LANGN_503,LANGN_514,LANGN_517,LANGN_520,LANGN_523,LANGN_527,LANGN_529,LANGN_531,LANGN_540,LANGN_547,LANGN_555,LANGN_561,LANGN_562,LANGN_563,LANGN_565,LANGN_566,LANGN_567,LANGN_600,LANGN_601,LANGN_602,LANGN_605,LANGN_606,LANGN_607,LANGN_608,LANGN_611,LANGN_614,LANGN_615,LANGN_616,LANGN_618,LANGN_619,LANGN_621,LANGN_622,LANGN_623,LANGN_624,LANGN_625,LANGN_626,LANGN_627,LANGN_628,LANGN_630,LANGN_631,LANGN_634,LANGN_635,LANGN_639,LANGN_640,LANGN_641,LANGN_642,LANGN_648,LANGN_650,LANGN_661,LANGN_662,LANGN_663,LANGN_665,LANGN_666,LANGN_667,LANGN_668,LANGN_669,LANGN_670,LANGN_673,LANGN_674,LANGN_675,LANGN_676,LANGN_677,LANGN_678,LANGN_800,LANGN_801,LANGN_802,LANGN_804,LANGN_805,LANGN_806,LANGN_807,LANGN_808,LANGN_809,LANGN_810,LANGN_811,LANGN

Now we'll copy the file to S3 for Amazon SageMaker's managed training to pickup.

In [16]:
# cell 14
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')

---

## End of Lab 1


---

## Training
Now we know that most of our features have skewed distributions, some are highly correlated with one another, and some appear to have non-linear relationships with our target variable.  Also, for targeting future prospects, good predictive accuracy is preferred to being able to explain why that prospect was targeted.  Taken together, these aspects make gradient boosted trees a good candidate algorithm.

There are several intricacies to understanding the algorithm, but at a high level, gradient boosted trees works by combining predictions from many simple models, each of which tries to address the weaknesses of the previous models.  By doing this the collection of simple models can actually outperform large, complex models.  Other Amazon SageMaker notebooks elaborate on gradient boosting trees further and how they differ from similar algorithms.

`xgboost` is an extremely popular, open-source package for gradient boosted trees.  It is computationally powerful, fully featured, and has been successfully used in many machine learning competitions.  Let's start with a simple `xgboost` model, trained using Amazon SageMaker's managed, distributed training framework.

First we'll need to specify the ECR container location for Amazon SageMaker's implementation of XGBoost.

In [17]:
# cell 15
container = sagemaker.image_uris.retrieve(region=boto3.Session().region_name, framework='xgboost', version='latest')

Then, because we're training with the CSV file format, we'll create `s3_input`s that our training function can use as a pointer to the files in S3, which also specify that the content type is CSV.

In [18]:
# cell 16
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')

First we'll need to specify training parameters to the estimator.  This includes:
1. The `xgboost` algorithm container
1. The IAM role to use
1. Training instance type and count
1. S3 location for output data
1. Algorithm hyperparameters

And then a `.fit()` function which specifies:
1. S3 location for output data.  In this case we have both a training and validation set which are passed in.

In [19]:
mlflow_arn = "arn:aws:sagemaker:us-west-2:986030204467:mlflow-tracking-server/capstone-ml-flow"
mlflow.set_tracking_uri(mlflow_arn)

In [20]:
# cell 17
sess = sagemaker.Session()

#with mlflow.start_run() as run:
xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100,
                        seed=42,  # Set fixed seed
                        seed_per_iteration=42,  # Ensures same randomness per iteration
                        early_stopping_rounds=10
                       )

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})
     # Log hyperparameters to MLflow
#    mlflow.log_params(xgb.hyperparameters())

INFO:sagemaker:Creating training-job with name: xgboost-2025-02-20-03-14-36-495


2025-02-20 03:14:36 Starting - Starting the training job...
2025-02-20 03:14:56 Starting - Preparing the instances for training...
2025-02-20 03:15:23 Downloading - Downloading input data...
2025-02-20 03:16:03 Downloading - Downloading the training image......
2025-02-20 03:16:54 Training - Training image download completed. Training in progress..Arguments: train
[2025-02-20:03:17:07:INFO] Running standalone xgboost training.
[2025-02-20:03:17:07:INFO] File size need to be processed in the node: 616.11mb. Available memory size in the node: 8551.0mb
[2025-02-20:03:17:07:INFO] Determined delimiter of CSV input is ','
[03:17:07] S3DistributionType set as FullyReplicated
[03:17:12] 414299x463 matrix with 191820437 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2025-02-20:03:17:12:INFO] Determined delimiter of CSV input is ','
[03:17:12] S3DistributionType set as FullyReplicated
[03:17:13] 118372x463 matrix with 54806236 entries loaded from /opt/ml/inpu

In [21]:

    # Log the trained model
    #mlflow.sagemaker.log_model(xgb, artifact_path="model", registered_model_name="PISA_XGBoost")

   # print("MLflow Run ID:", run.info.run_id)

MLflow Run ID: 259e2f904eea419a90f0afd7dee17e3a


# Explain the Model using Clarify

In [22]:
from datetime import datetime

session = sagemaker.Session()
model_name = "DEMO-pisa-2022-clarify-model-{}".format(datetime.now().strftime("%d-%m-%Y-%H-%M-%S"))
model = xgb.create_model(name=model_name)
container_def = model.prepare_container_def()
session.create_model(model_name, role, container_def)

INFO:sagemaker:Creating model with name: DEMO-pisa-2022-clarify-model-20-02-2025-03-23-58


'DEMO-pisa-2022-clarify-model-20-02-2025-03-23-58'

# Explaining predictions

In [23]:
train_data.head(5)

,MATH_Proficient,CNTSCHID,CNTSTUID,SISCO,ST347Q01JA,ST347Q02JA,ST349Q01JA_0,ST349Q01JA_1,ST349Q01JA_2,ST349Q01JA_3,ST349Q01JA_4,ST350Q01JA,ST356Q01JA,ST322Q01JA,ST322Q02JA,ST322Q03JA,ST322Q04JA,ST322Q06JA,ST322Q07JA,DURECEC,EFFORT1,EFFORT2,ST259Q01JA,WB164Q01HA,ST004D01T,GRADE,REPEAT,EXPECEDU,ICTAVSCH,ICTAVHOM,ICTDISTR,IMMIG,TARDYSD,ST226Q01JA,ST016Q01NA,MISSSC,Option_UH,OECD,PAREDINT,BMMJ1,BFMJ2,WB163Q06HA,WB163Q07HA,ST230Q01JA,SKIPPING,IC180Q01JA,IC180Q08JA,ST059Q02JA,ST296Q04JA,WB176Q01HA,STUDYHMW,IC184Q01JA,IC184Q02JA,IC184Q03JA,IC184Q04JA,ST059Q01TA,ST296Q01JA,ST272Q01JA,ST268Q01JA,ST268Q04JA,ST268Q07JA,ST293Q04JA,ST297Q01JA,ST297Q03JA,ST297Q05JA,ST297Q06JA,ST297Q07JA,ST297Q09JA,WB165Q01HA,WB166Q01HA,WB166Q02HA,WB166Q03HA,WB166Q04HA,ST258Q01JA,ST294Q01JA,ST295Q01JA,WB150Q01HA,WB156Q01HA,WB158Q01HA,WB160Q01HA,WB161Q01HA,WB171Q01HA,WB171Q02HA,WB171Q03HA,WB171Q04HA,WB172Q01HA,WB173Q01HA,WB173Q02HA,WB173Q03HA,WB173Q04HA,WB177Q01HA,WB177Q02HA,WB177Q03HA,WB177Q04HA,WB032Q01NA,WB032Q02NA,WB031Q01NA,EXERPRAC,STUBMI,ESCS,WORKPAY,WORKHOME,SC001Q01TA,SC211Q01JA,SC211Q02JA,SC211Q03JA,SC211Q04JA,SC211Q05JA,SC211Q06JA,SC209Q04JA,SC209Q05JA,SC209Q06JA,SC037Q11JA,SC183Q02JA,SC183Q03JA,SC183Q04JA,SC175Q01JA,SC177Q01JA_1,SC177Q01JA_2,SC177Q01JA_3,SC177Q02JA_1,SC177Q02JA_2,SC177Q02JA_3,SC177Q03JA_1,SC177Q03JA_2,SC177Q03JA_3,SC188Q01JA,SC188Q02JA,SC188Q03JA,SC188Q04JA,SC188Q05JA,SC188Q06JA,SC188Q07JA,SC188Q08JA,SC188Q09JA,SC188Q10JA,SC188Q11JA,SC198Q01JA,SC198Q02JA,SC198Q03JA,SC178Q01JA,SC178Q02JA,SC180Q01JA,SC189Q02WA,SC189Q03WA,SC189Q04WA,SMRATIO,MCLSIZE,MACTIV,MATHEXC_0,MATHEXC_1,MATHEXC_2,MATHEXC_3,ABGMATH,SC064Q05WA,SC064Q06WA,SC064Q01TA,SC064Q02TA,SC064Q04NA,SC064Q03TA,SC064Q07WA,SC213Q01JA,SC213Q02JA,SC037Q01TA,SC037Q02TA,SC037Q03TA,SC037Q04TA,SC037Q05NA,SC037Q06NA,SC037Q07TA,SC037Q09TA,SC200Q01JA,SC200Q02JA,SC200Q03JA,SC200Q04JA,SC224Q01JA,RATCMP1,RATCMP2,RATTAB,SCHSEL,SCHLTYPE_1,SCHLTYPE_2,SCHLTYPE_3,SC034Q01NA,SC034Q02NA,SC034Q03TA,SC034Q04TA,SC195Q01JA,SC195Q02JA,SC195Q03JA,SC195Q04JA,SC042Q01TA,SC042Q02TA,SC214Q01JA,SC214Q02JA,SC214Q03JA,SC215Q01JA,SC215Q02JA,SC215Q03JA,SC215Q04JA,SC215Q05JA,SC215Q06JA,SC215Q07JA,SC215Q08JA,SC216Q06JA,SC216Q07JA,SC216Q08JA,SC216Q09JA,SC217Q01JA,SC217Q02JA,SC217Q03JA,SC217Q04JA,SC217Q05JA,SC217Q06JA,SC217Q07JA,SC217Q08JA,SC217Q10JA,SC218Q01JA,SC219Q01JA,SC220Q01JA,SC221Q01JA,SC221Q02JA,SC221Q03JA,SC221Q04JA,SCSUPRTED,SCSUPRT,SC212Q01JA,SC212Q02JA,SC212Q03JA,SC037Q08TA,SC032Q01TA,SC032Q02TA,SC032Q03TA,SC032Q04TA,LANGN_105,LANGN_108,LANGN_112,LANGN_113,LANGN_118,LANGN_121,LANGN_130,LANGN_133,LANGN_137,LANGN_140,LANGN_147,LANGN_148,LANGN_150,LANGN_154,LANGN_156,LANGN_160,LANGN_170,LANGN_195,LANGN_200,LANGN_202,LANGN_204,LANGN_232,LANGN_237,LANGN_244,LANGN_246,LANGN_254,LANGN_258,LANGN_263,LANGN_264,LANGN_266,LANGN_272,LANGN_273,LANGN_275,LANGN_286,LANGN_301,LANGN_313,LANGN_316,LANGN_317,LANGN_322,LANGN_325,LANGN_327,LANGN_329,LANGN_338,LANGN_340,LANGN_344,LANGN_351,LANGN_358,LANGN_363,LANGN_369,LANGN_371,LANGN_375,LANGN_379,LANGN_381,LANGN_382,LANGN_383,LANGN_404,LANGN_409,LANGN_415,LANGN_420,LANGN_422,LANGN_428,LANGN_434,LANGN_442,LANGN_449,LANGN_451,LANGN_463,LANGN_465,LANGN_467,LANGN_471,LANGN_472,LANGN_474,LANGN_492,LANGN_493,LANGN_494,LANGN_495,LANGN_496,LANGN_500,LANGN_503,LANGN_514,LANGN_517,LANGN_520,LANGN_523,LANGN_527,LANGN_529,LANGN_531,LANGN_540,LANGN_547,LANGN_555,LANGN_561,LANGN_562,LANGN_563,LANGN_565,LANGN_566,LANGN_567,LANGN_600,LANGN_601,LANGN_602,LANGN_605,LANGN_606,LANGN_607,LANGN_608,LANGN_611,LANGN_614,LANGN_615,LANGN_616,LANGN_618,LANGN_619,LANGN_621,LANGN_622,LANGN_623,LANGN_624,LANGN_625,LANGN_626,LANGN_627,LANGN_628,LANGN_630,LANGN_631,LANGN_634,LANGN_635,LANGN_639,LANGN_640,LANGN_641,LANGN_642,LANGN_648,LANGN_650,LANGN_661,LANGN_662,LANGN_663,LANGN_665,LANGN_666,LANGN_667,LANGN_668,LANGN_669,LANGN_670,LANGN_673,LANGN_674,LANGN_675,LANGN_676,LANGN_677,LANGN_678,LANGN_800,LANGN_801,LANGN_802,LANGN_804,LANGN_805,LANGN_806,LANGN_807,LANGN_808,LANGN_809,LANGN_810,LANGN_811,LANGN

In [24]:
validation_data.head(5)

,MATH_Proficient,CNTSCHID,CNTSTUID,SISCO,ST347Q01JA,ST347Q02JA,ST349Q01JA_0,ST349Q01JA_1,ST349Q01JA_2,ST349Q01JA_3,ST349Q01JA_4,ST350Q01JA,ST356Q01JA,ST322Q01JA,ST322Q02JA,ST322Q03JA,ST322Q04JA,ST322Q06JA,ST322Q07JA,DURECEC,EFFORT1,EFFORT2,ST259Q01JA,WB164Q01HA,ST004D01T,GRADE,REPEAT,EXPECEDU,ICTAVSCH,ICTAVHOM,ICTDISTR,IMMIG,TARDYSD,ST226Q01JA,ST016Q01NA,MISSSC,Option_UH,OECD,PAREDINT,BMMJ1,BFMJ2,WB163Q06HA,WB163Q07HA,ST230Q01JA,SKIPPING,IC180Q01JA,IC180Q08JA,ST059Q02JA,ST296Q04JA,WB176Q01HA,STUDYHMW,IC184Q01JA,IC184Q02JA,IC184Q03JA,IC184Q04JA,ST059Q01TA,ST296Q01JA,ST272Q01JA,ST268Q01JA,ST268Q04JA,ST268Q07JA,ST293Q04JA,ST297Q01JA,ST297Q03JA,ST297Q05JA,ST297Q06JA,ST297Q07JA,ST297Q09JA,WB165Q01HA,WB166Q01HA,WB166Q02HA,WB166Q03HA,WB166Q04HA,ST258Q01JA,ST294Q01JA,ST295Q01JA,WB150Q01HA,WB156Q01HA,WB158Q01HA,WB160Q01HA,WB161Q01HA,WB171Q01HA,WB171Q02HA,WB171Q03HA,WB171Q04HA,WB172Q01HA,WB173Q01HA,WB173Q02HA,WB173Q03HA,WB173Q04HA,WB177Q01HA,WB177Q02HA,WB177Q03HA,WB177Q04HA,WB032Q01NA,WB032Q02NA,WB031Q01NA,EXERPRAC,STUBMI,ESCS,WORKPAY,WORKHOME,SC001Q01TA,SC211Q01JA,SC211Q02JA,SC211Q03JA,SC211Q04JA,SC211Q05JA,SC211Q06JA,SC209Q04JA,SC209Q05JA,SC209Q06JA,SC037Q11JA,SC183Q02JA,SC183Q03JA,SC183Q04JA,SC175Q01JA,SC177Q01JA_1,SC177Q01JA_2,SC177Q01JA_3,SC177Q02JA_1,SC177Q02JA_2,SC177Q02JA_3,SC177Q03JA_1,SC177Q03JA_2,SC177Q03JA_3,SC188Q01JA,SC188Q02JA,SC188Q03JA,SC188Q04JA,SC188Q05JA,SC188Q06JA,SC188Q07JA,SC188Q08JA,SC188Q09JA,SC188Q10JA,SC188Q11JA,SC198Q01JA,SC198Q02JA,SC198Q03JA,SC178Q01JA,SC178Q02JA,SC180Q01JA,SC189Q02WA,SC189Q03WA,SC189Q04WA,SMRATIO,MCLSIZE,MACTIV,MATHEXC_0,MATHEXC_1,MATHEXC_2,MATHEXC_3,ABGMATH,SC064Q05WA,SC064Q06WA,SC064Q01TA,SC064Q02TA,SC064Q04NA,SC064Q03TA,SC064Q07WA,SC213Q01JA,SC213Q02JA,SC037Q01TA,SC037Q02TA,SC037Q03TA,SC037Q04TA,SC037Q05NA,SC037Q06NA,SC037Q07TA,SC037Q09TA,SC200Q01JA,SC200Q02JA,SC200Q03JA,SC200Q04JA,SC224Q01JA,RATCMP1,RATCMP2,RATTAB,SCHSEL,SCHLTYPE_1,SCHLTYPE_2,SCHLTYPE_3,SC034Q01NA,SC034Q02NA,SC034Q03TA,SC034Q04TA,SC195Q01JA,SC195Q02JA,SC195Q03JA,SC195Q04JA,SC042Q01TA,SC042Q02TA,SC214Q01JA,SC214Q02JA,SC214Q03JA,SC215Q01JA,SC215Q02JA,SC215Q03JA,SC215Q04JA,SC215Q05JA,SC215Q06JA,SC215Q07JA,SC215Q08JA,SC216Q06JA,SC216Q07JA,SC216Q08JA,SC216Q09JA,SC217Q01JA,SC217Q02JA,SC217Q03JA,SC217Q04JA,SC217Q05JA,SC217Q06JA,SC217Q07JA,SC217Q08JA,SC217Q10JA,SC218Q01JA,SC219Q01JA,SC220Q01JA,SC221Q01JA,SC221Q02JA,SC221Q03JA,SC221Q04JA,SCSUPRTED,SCSUPRT,SC212Q01JA,SC212Q02JA,SC212Q03JA,SC037Q08TA,SC032Q01TA,SC032Q02TA,SC032Q03TA,SC032Q04TA,LANGN_105,LANGN_108,LANGN_112,LANGN_113,LANGN_118,LANGN_121,LANGN_130,LANGN_133,LANGN_137,LANGN_140,LANGN_147,LANGN_148,LANGN_150,LANGN_154,LANGN_156,LANGN_160,LANGN_170,LANGN_195,LANGN_200,LANGN_202,LANGN_204,LANGN_232,LANGN_237,LANGN_244,LANGN_246,LANGN_254,LANGN_258,LANGN_263,LANGN_264,LANGN_266,LANGN_272,LANGN_273,LANGN_275,LANGN_286,LANGN_301,LANGN_313,LANGN_316,LANGN_317,LANGN_322,LANGN_325,LANGN_327,LANGN_329,LANGN_338,LANGN_340,LANGN_344,LANGN_351,LANGN_358,LANGN_363,LANGN_369,LANGN_371,LANGN_375,LANGN_379,LANGN_381,LANGN_382,LANGN_383,LANGN_404,LANGN_409,LANGN_415,LANGN_420,LANGN_422,LANGN_428,LANGN_434,LANGN_442,LANGN_449,LANGN_451,LANGN_463,LANGN_465,LANGN_467,LANGN_471,LANGN_472,LANGN_474,LANGN_492,LANGN_493,LANGN_494,LANGN_495,LANGN_496,LANGN_500,LANGN_503,LANGN_514,LANGN_517,LANGN_520,LANGN_523,LANGN_527,LANGN_529,LANGN_531,LANGN_540,LANGN_547,LANGN_555,LANGN_561,LANGN_562,LANGN_563,LANGN_565,LANGN_566,LANGN_567,LANGN_600,LANGN_601,LANGN_602,LANGN_605,LANGN_606,LANGN_607,LANGN_608,LANGN_611,LANGN_614,LANGN_615,LANGN_616,LANGN_618,LANGN_619,LANGN_621,LANGN_622,LANGN_623,LANGN_624,LANGN_625,LANGN_626,LANGN_627,LANGN_628,LANGN_630,LANGN_631,LANGN_634,LANGN_635,LANGN_639,LANGN_640,LANGN_641,LANGN_642,LANGN_648,LANGN_650,LANGN_661,LANGN_662,LANGN_663,LANGN_665,LANGN_666,LANGN_667,LANGN_668,LANGN_669,LANGN_670,LANGN_673,LANGN_674,LANGN_675,LANGN_676,LANGN_677,LANGN_678,LANGN_800,LANGN_801,LANGN_802,LANGN_804,LANGN_805,LANGN_806,LANGN_807,LANGN_808,LANGN_809,LANGN_810,LANGN_811,LANGN

In [25]:
test_data.head(5)

,MATH_Proficient,CNT,CNTSCHID,CNTSTUID,SISCO,ST347Q01JA,ST347Q02JA,ST349Q01JA_0,ST349Q01JA_1,ST349Q01JA_2,ST349Q01JA_3,ST349Q01JA_4,ST350Q01JA,ST356Q01JA,ST322Q01JA,ST322Q02JA,ST322Q03JA,ST322Q04JA,ST322Q06JA,ST322Q07JA,DURECEC,EFFORT1,EFFORT2,ST259Q01JA,WB164Q01HA,ST004D01T,GRADE,REPEAT,EXPECEDU,ICTAVSCH,ICTAVHOM,ICTDISTR,IMMIG,TARDYSD,ST226Q01JA,ST016Q01NA,MISSSC,Option_UH,OECD,PAREDINT,BMMJ1,BFMJ2,WB163Q06HA,WB163Q07HA,ST230Q01JA,SKIPPING,IC180Q01JA,IC180Q08JA,ST059Q02JA,ST296Q04JA,WB176Q01HA,STUDYHMW,IC184Q01JA,IC184Q02JA,IC184Q03JA,IC184Q04JA,ST059Q01TA,ST296Q01JA,ST272Q01JA,ST268Q01JA,ST268Q04JA,ST268Q07JA,ST293Q04JA,ST297Q01JA,ST297Q03JA,ST297Q05JA,ST297Q06JA,ST297Q07JA,ST297Q09JA,WB165Q01HA,WB166Q01HA,WB166Q02HA,WB166Q03HA,WB166Q04HA,ST258Q01JA,ST294Q01JA,ST295Q01JA,WB150Q01HA,WB156Q01HA,WB158Q01HA,WB160Q01HA,WB161Q01HA,WB171Q01HA,WB171Q02HA,WB171Q03HA,WB171Q04HA,WB172Q01HA,WB173Q01HA,WB173Q02HA,WB173Q03HA,WB173Q04HA,WB177Q01HA,WB177Q02HA,WB177Q03HA,WB177Q04HA,WB032Q01NA,WB032Q02NA,WB031Q01NA,EXERPRAC,STUBMI,ESCS,WORKPAY,WORKHOME,SC001Q01TA,SC211Q01JA,SC211Q02JA,SC211Q03JA,SC211Q04JA,SC211Q05JA,SC211Q06JA,SC209Q04JA,SC209Q05JA,SC209Q06JA,SC037Q11JA,SC183Q02JA,SC183Q03JA,SC183Q04JA,SC175Q01JA,SC177Q01JA_1,SC177Q01JA_2,SC177Q01JA_3,SC177Q02JA_1,SC177Q02JA_2,SC177Q02JA_3,SC177Q03JA_1,SC177Q03JA_2,SC177Q03JA_3,SC188Q01JA,SC188Q02JA,SC188Q03JA,SC188Q04JA,SC188Q05JA,SC188Q06JA,SC188Q07JA,SC188Q08JA,SC188Q09JA,SC188Q10JA,SC188Q11JA,SC198Q01JA,SC198Q02JA,SC198Q03JA,SC178Q01JA,SC178Q02JA,SC180Q01JA,SC189Q02WA,SC189Q03WA,SC189Q04WA,SMRATIO,MCLSIZE,MACTIV,MATHEXC_0,MATHEXC_1,MATHEXC_2,MATHEXC_3,ABGMATH,SC064Q05WA,SC064Q06WA,SC064Q01TA,SC064Q02TA,SC064Q04NA,SC064Q03TA,SC064Q07WA,SC213Q01JA,SC213Q02JA,SC037Q01TA,SC037Q02TA,SC037Q03TA,SC037Q04TA,SC037Q05NA,SC037Q06NA,SC037Q07TA,SC037Q09TA,SC200Q01JA,SC200Q02JA,SC200Q03JA,SC200Q04JA,SC224Q01JA,RATCMP1,RATCMP2,RATTAB,SCHSEL,SCHLTYPE_1,SCHLTYPE_2,SCHLTYPE_3,SC034Q01NA,SC034Q02NA,SC034Q03TA,SC034Q04TA,SC195Q01JA,SC195Q02JA,SC195Q03JA,SC195Q04JA,SC042Q01TA,SC042Q02TA,SC214Q01JA,SC214Q02JA,SC214Q03JA,SC215Q01JA,SC215Q02JA,SC215Q03JA,SC215Q04JA,SC215Q05JA,SC215Q06JA,SC215Q07JA,SC215Q08JA,SC216Q06JA,SC216Q07JA,SC216Q08JA,SC216Q09JA,SC217Q01JA,SC217Q02JA,SC217Q03JA,SC217Q04JA,SC217Q05JA,SC217Q06JA,SC217Q07JA,SC217Q08JA,SC217Q10JA,SC218Q01JA,SC219Q01JA,SC220Q01JA,SC221Q01JA,SC221Q02JA,SC221Q03JA,SC221Q04JA,SCSUPRTED,SCSUPRT,SC212Q01JA,SC212Q02JA,SC212Q03JA,SC037Q08TA,SC032Q01TA,SC032Q02TA,SC032Q03TA,SC032Q04TA,LANGN_105,LANGN_108,LANGN_112,LANGN_113,LANGN_118,LANGN_121,LANGN_130,LANGN_133,LANGN_137,LANGN_140,LANGN_147,LANGN_148,LANGN_150,LANGN_154,LANGN_156,LANGN_160,LANGN_170,LANGN_195,LANGN_200,LANGN_202,LANGN_204,LANGN_232,LANGN_237,LANGN_244,LANGN_246,LANGN_254,LANGN_258,LANGN_263,LANGN_264,LANGN_266,LANGN_272,LANGN_273,LANGN_275,LANGN_286,LANGN_301,LANGN_313,LANGN_316,LANGN_317,LANGN_322,LANGN_325,LANGN_327,LANGN_329,LANGN_338,LANGN_340,LANGN_344,LANGN_351,LANGN_358,LANGN_363,LANGN_369,LANGN_371,LANGN_375,LANGN_379,LANGN_381,LANGN_382,LANGN_383,LANGN_404,LANGN_409,LANGN_415,LANGN_420,LANGN_422,LANGN_428,LANGN_434,LANGN_442,LANGN_449,LANGN_451,LANGN_463,LANGN_465,LANGN_467,LANGN_471,LANGN_472,LANGN_474,LANGN_492,LANGN_493,LANGN_494,LANGN_495,LANGN_496,LANGN_500,LANGN_503,LANGN_514,LANGN_517,LANGN_520,LANGN_523,LANGN_527,LANGN_529,LANGN_531,LANGN_540,LANGN_547,LANGN_555,LANGN_561,LANGN_562,LANGN_563,LANGN_565,LANGN_566,LANGN_567,LANGN_600,LANGN_601,LANGN_602,LANGN_605,LANGN_606,LANGN_607,LANGN_608,LANGN_611,LANGN_614,LANGN_615,LANGN_616,LANGN_618,LANGN_619,LANGN_621,LANGN_622,LANGN_623,LANGN_624,LANGN_625,LANGN_626,LANGN_627,LANGN_628,LANGN_630,LANGN_631,LANGN_634,LANGN_635,LANGN_639,LANGN_640,LANGN_641,LANGN_642,LANGN_648,LANGN_650,LANGN_661,LANGN_662,LANGN_663,LANGN_665,LANGN_666,LANGN_667,LANGN_668,LANGN_669,LANGN_670,LANGN_673,LANGN_674,LANGN_675,LANGN_676,LANGN_677,LANGN_678,LANGN_800,LANGN_801,LANGN_802,LANGN_804,LANGN_805,LANGN_806,LANGN_807,LANGN_808,LANGN_809,LANGN_810,LANGN_811,L

In [26]:
test_data = test_data.drop(["CNT"], axis=1)
test_features = test_data.drop(["MATH_Proficient"], axis=1)
test_target = test_data["MATH_Proficient"]
test_features.to_csv("test_features.csv", index=False, header=False)

In [27]:
from sagemaker import clarify
clarify_processor = clarify.SageMakerClarifyProcessor(
    role=role, instance_count=1, instance_type="ml.m5.2xlarge", sagemaker_session=session
)
model_config = clarify.ModelConfig(
    model_name=model_name,
    instance_type="ml.m5.2xlarge",
    instance_count=1,
    accept_type="text/csv",
    content_type="text/csv",
)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: 1.0.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [28]:
from sagemaker.s3 import S3Downloader
# Download data from S3 to local instance
local_path = S3Downloader.download('s3://{}/{}/train'.format(bucket, prefix), './tmp/train_data')

In [29]:
local_path

['./tmp/train_data/train.csv']

In [30]:
# Load and sample
full_data = pd.read_csv('./tmp/train_data/train.csv', header=None)
sampled_data = full_data.sample(n=1500)  # Adjust the sample size as needed

# Save sampled data back to S3
sampled_path = 'sampled_train_data.csv'
sampled_data.to_csv(sampled_path, index=False)

from sagemaker.s3 import S3Uploader
sampled_s3_uri = S3Uploader.upload(sampled_path, 's3://{}/{}/sampled_train'.format(bucket, prefix))

In [31]:
sampled_data.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463
280973,1,70200139,70204716,1.0,NaN,NaN,0,0,0,0,0,NaN,NaN,NaN,1.0,2.0,1.0,1.0,3.0,3.0,6.0,8.0,NaN,NaN,2.0,0.0,0.0,8.0,7.0,6.0,NaN,2.0,0.0,1.0,NaN,0.0,0,0,16.0,74.79,72.94,NaN,NaN,3.0,1.0,2.0,1.0,31.0,2.0,NaN,0.0,1.0,2.0,1.0,1.0,6.0,1.0,NaN,4.0,4.0,3.0,NaN,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,2.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,1.7249,0.0,0.0,5.0,1.0,NaN,NaN,2.0,2.0,NaN,2.0,2.0,2.0,1.0,1.0,1.0,1.0,40.0,0,1,0,0,0,1,0,0,1,NaN,NaN,4.0,4.0,NaN,NaN,3.0,3.0,3.0,3.0,2.0,2.0,1.0,1.0,95.0,5.0,1.0,2.0,1.0,1.0,70.1067,28.0,2.0,0,0,0,1,2.0,10.0,5.0,10.0,20.0,NaN,1.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.6763,1.0,0.0000,3.0,0,0,1,3.0,2.0,3.0,2.0,2.0,2.0,3.0,2.0,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
77741,0,72400105,72410464,NaN,NaN,NaN,0,0,0,0,0,NaN,NaN,NaN,1.0,5.0,2.0,3.0,2.0,1.0,1.0,2.0,10.0,3.0,1.0,0.0,0.0,9.0,7.0,6.0,9.0,NaN,2.0,1.0,10.0,NaN,0,1,9.0,NaN,NaN,3.0,2.0,3.0,1.0,2.0,2.0,11.0,1.0,NaN,1.0,2.0,2.0,2.0,2.0,4.0,1.0,6.0,2.0,2.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,1.0,1.0,2.0,NaN,6.0,6.0,1.0,2.0,4.0,2.0,2.0,2.0,1.0,1.0,1.0,3.0,1.0,4.0,1.0,4.0,NaN,NaN,NaN,NaN,6.0,7.0,3.0,10.0,21.78,-0.9040,0.0,10.0,5.0,13.0,6.0,3.0,2.0,9.0,0.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,60.0,0,0,0,0,0,0,0,0,0,3.0,4.0,4.0,3.0,2.0,2.0,2.0,3.0,3.0,4.0,1.0,2.0,2.0,1.0,62.0,38.0,2.0,2.0,1.0,1.0,100.0000,28.0,2.0,0,0,0,0,3.0,40.0,74.0,49.0,80.0,12.0,10.0,3.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0000,1.0,0.0711,2.0,0,0,1,2.0,1.0,5.0,5.0,2.0,1.0,4.0,5.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [32]:
shap_config = clarify.SHAPConfig(
    baseline=[test_features.iloc[0].values.tolist()],
    num_samples=3000,
    agg_method="mean_abs",
    save_local_shap_values=True
)

explainability_output_path = "s3://{}/{}/clarify-explainability".format(bucket, prefix)
explainability_data_config = clarify.DataConfig(
    #s3_data_input_path='s3://{}/{}/train'.format(bucket, prefix),
    s3_data_input_path=sampled_s3_uri,
    s3_output_path=explainability_output_path,
    label='MATH_Proficient',
    headers=train_data.columns.to_list(),
    dataset_type="text/csv",
)

In [33]:
clarify_processor.run_explainability(
    data_config=explainability_data_config,
    model_config=model_config,
    explainability_config=shap_config
)

INFO:sagemaker.clarify:Analysis Config: {'dataset_type': 'text/csv', 'headers': ['MATH_Proficient', 'CNTSCHID', 'CNTSTUID', 'SISCO', 'ST347Q01JA', 'ST347Q02JA', 'ST349Q01JA_0', 'ST349Q01JA_1', 'ST349Q01JA_2', 'ST349Q01JA_3', 'ST349Q01JA_4', 'ST350Q01JA', 'ST356Q01JA', 'ST322Q01JA', 'ST322Q02JA', 'ST322Q03JA', 'ST322Q04JA', 'ST322Q06JA', 'ST322Q07JA', 'DURECEC', 'EFFORT1', 'EFFORT2', 'ST259Q01JA', 'WB164Q01HA', 'ST004D01T', 'GRADE', 'REPEAT', 'EXPECEDU', 'ICTAVSCH', 'ICTAVHOM', 'ICTDISTR', 'IMMIG', 'TARDYSD', 'ST226Q01JA', 'ST016Q01NA', 'MISSSC', 'Option_UH', 'OECD', 'PAREDINT', 'BMMJ1', 'BFMJ2', 'WB163Q06HA', 'WB163Q07HA', 'ST230Q01JA', 'SKIPPING', 'IC180Q01JA', 'IC180Q08JA', 'ST059Q02JA', 'ST296Q04JA', 'WB176Q01HA', 'STUDYHMW', 'IC184Q01JA', 'IC184Q02JA', 'IC184Q03JA', 'IC184Q04JA', 'ST059Q01TA', 'ST296Q01JA', 'ST272Q01JA', 'ST268Q01JA', 'ST268Q04JA', 'ST268Q07JA', 'ST293Q04JA', 'ST297Q01JA', 'ST297Q03JA', 'ST297Q05JA', 'ST297Q06JA', 'ST297Q07JA', 'ST297Q09JA', 'WB165Q01HA', 'WB166Q01

.................sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
We are not in a supported iso region, /bin/sh exiting gracefully with no changes.
INFO:sagemaker-clarify-processing:Starting SageMaker Clarify Processing job
INFO:analyzer.data_loading.data_loader_util:Analysis config path: /opt/ml/processing/input/config/analysis_config.json
INFO:analyzer.data_loading.data_loader_util:Analysis result path: /opt/ml/processing/output
INFO:analyzer.data_loading.data_loader_util:This host is algo-1.
INFO:analyzer.data_loading.data_loader_util:This host is the leader.
INFO:analyzer.data_loading.data_loader_util:Number of hosts in the cluster is 1.
INFO:sagemaker-clarify-processing:Running Python / Pandas based analyzer.
INFO:analyzer.data_loading.data_loader_factory:Dataset type: text/csv uri: /opt/ml/processing/input/data
INFO:sagemaker-clarify-